**Imports**

* ***init_chat_model***: initializes a chat-based LLM
* ***initialize_agent***: creates an agent, system that uses an LLM plus tools to perform reasining and actions
* ***AgentType***: Enum to specify the type of agent
* ***Tool***: Represents a tool (an action the agent can take), such as a web search, calculator, or database query
* ***TavilySearchResults***: A community-contributed tool for querying Tavily Search API (web search for LLMs).
* ***ConversationBufferMemory***: Memory that stores the entire conversation history
* ***HumanMessage***: A message from the user
* ***AIMessage***: A message from the AI

In [1]:
from langchain.chat_models import init_chat_model
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.chat_models import init_chat_model
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage

I set the LangSmith API (authenticate with LangSmith for tracing and analytics) key and the Tavily API key (perform web searches)

- With these we can run LangChain agents with tools
- Log and monitor runs in LangSmith
- Use the Tavily Search tool for web lookups.

In [2]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true" # Enable LangSmith tracing
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("LANGSMITH_API_KEY: \n")

os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY: \n")

We create a model using Llama3 model, provided by Ollama (downloaded in my computer).

The settings for the model is 150 tokens and control randomness with temperature, set to 0, what means deterministic output.

In [3]:
# Simple model setup
model = init_chat_model(
    "llama3",
    model_provider="ollama",
    model_kwargs={"max_tokens": 150, "temperature": 0}
)

I create a memeory module for tracking the chat history.

When True, memory returns the conversation as a list of structured message objects. Preserves role information-

In [4]:
# Simple memory
memory = ConversationBufferMemory(return_messages=True)


C:\Users\nerea\AppData\Local\Temp\ipykernel_11552\722834312.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


We create a minimal LangChain agent that uses the model, keeps track of conversation history and manually builds the propmpt.

* **CHAT function** -> Gets the conversation history, create a base system prompt, append the last 6 messages, add the new user message, calls the model, saves the turn to memory and return the response.
* **SHOW_MEMORY fuction** -> Prints all stored messages in the conversation memory for debugging

In [5]:
class SimpleAgent:
    def __init__(self, model, memory):
        self.model = model
        self.memory = memory
    
    def chat(self, message):
        # Get conversation history
        history = self.memory.chat_memory.messages
        
        # Create context with history
        context = "You are a helpful assistant. Remember information from our conversation.\n\n"
        
        # Add conversation history
        for msg in history[-6:]:  # Last 6 messages only
            if isinstance(msg, HumanMessage):
                context += f"Human: {msg.content}\n"
            elif isinstance(msg, AIMessage):
                context += f"Assistant: {msg.content}\n"
        
        # Add current message
        context += f"Human: {message}\nAssistant:"
        
        try:
            # Get response
            response = self.model.invoke(context)
            response_text = response.content
            
            # Save to memory
            self.memory.chat_memory.add_user_message(message)
            self.memory.chat_memory.add_ai_message(response_text)
            
            return response_text
            
        except Exception as e:
            return f"Error: {str(e)}"
    
    def show_memory(self):
        messages = self.memory.chat_memory.messages
        print(f"Memory has {len(messages)} messages:")
        for i, msg in enumerate(messages):
            msg_type = "Human" if isinstance(msg, HumanMessage) else "AI"
            print(f"{i}: {msg_type}: {msg.content}")

In [6]:
# Create and test the simple agent
print("=== Creating Simple Agent ===")
simple_agent = SimpleAgent(model, memory)

# Test 1: Greeting with name
print("\n1. Greeting with name...")
response1 = simple_agent.chat("Hello! My name is Nerea. Please remember this for our conversation.")
print(f"Agent: {response1}")

=== Creating Simple Agent ===

1. Greeting with name...
Agent: Nice to meet you, Nerea! I'll make sure to remember your name throughout our conversation. Got it!


In [7]:
print("\n2. Testing memory...")
response2 = simple_agent.chat("What is my name?")
print(f"Agent: {response2}")


2. Testing memory...
Agent: Your name is Nerea!


In [8]:
# Show memory contents
print("\n=== Memory Contents ===")
simple_agent.show_memory()


=== Memory Contents ===
Memory has 4 messages:
0: Human: Hello! My name is Nerea. Please remember this for our conversation.
1: AI: Nice to meet you, Nerea! I'll make sure to remember your name throughout our conversation. Got it!
2: Human: What is my name?
3: AI: Your name is Nerea!


In [9]:
# Test 3: Simple conversation
print("\n3. Simple conversation...")
response3 = simple_agent.chat("I live in denmark.")
response4 = simple_agent.chat("What was my name again?")
print(f"Agent: {response3}")
print(f"Agent: {response4}")


3. Simple conversation...
Agent: You live in Denmark! I've got that note down!
Agent: Your name is Nerea!


In [10]:
response5 = simple_agent.chat("Tell me a short description about me with the things I have told you.")
print(f"Agent: {response5}")

Agent: Let me see... Based on our conversation, here's what I know about you:

You're named Nerea! And you live in Denmark!

That's all I've got so far!
